The current setup allows to execute PySpark operations on CERN Hadoop and Spark clusters. This notebook illustrates the use of Spark in SWAN to analyze the monitoring data available on HDFS and plots a heatmap of loadAvg across machines in a particular service.

### Connect to the cluster

To connect to a cluster, click on the star button on the top and follow the instructions
* The star button only appears if you have selected a SPARK cluster in the configuration
* The star button is active after the notebook kernel is ready

### Import necessary spark and python stuff

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data
*path_on_hdfs_to_your_data*

In [3]:
df = spark.read.json("/user/rucio01/traces/traces.2018-04*")

### check the structure

In [4]:
# df.printSchema()

### Filter for loadAvg
_https://metricmgr.cern.ch/metric/20002/_

In [5]:
df.head()

Row(account=u'pilot', appid=None, catStart=None, clientState=u'DONE', dataset=u'', datasetName=None, datasetScope=u'', dataset_scope=None, duid=None, eventType=u'download', eventVersion=u'1.14.11', filename=u'EventService_premerge_3886111819_13607623-3886111819-13041798597-3795-10.tar', filesize=None, guid=None, hostname=u'b6ac037cc1.cern.ch', ip=None, localSite=None, name=None, pq=None, protocol=u's3', relativeStart=None, remoteSite=u'CERN-PROD_ES', scope=u'transient', stateReason=None, suspicious=None, taskid=None, timeEnd=None, timeStart=None, traceId=u'e68c1b1bbee84d5a96dcfc2f38dc26bb', traceIp=u'::ffff:188.184.146.18', traceTimeentry=u'2018-03-31T22:00:00.005599', traceTimeentryUnix=1522533600.005599, transferEnd=1522533599.975359, transferStart=1522533588.042424, url=None, usr=None, usrdn=None, uuid=u'53cf181ec38b4e2a8885b2a81109d505', validateStart=None, version=None)

### Create temporary table view

In [6]:
# body_schema = spark.read.json(df_loadAvg.rdd.map(lambda row: row.body)).schema
df.createOrReplaceTempView("traces")

### Use PySpark to query download events and split dataset DID into new features

In [7]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [8]:
df_ps = spark.sql("SELECT account, clientState, dataset, eventType, eventVersion, filename, filesize, remoteSite, traceTimeentryUnix, count(*) \
                               FROM traces \
                               WHERE dataset != 'None' AND dataset != '' AND dataset != '[null]' AND (dataset LIKE 'data%' OR dataset LIKE 'mc%')\
                               GROUP BY account, clientState, dataset, datasetName, datasetScope, dataset_scope, eventType, eventVersion, filename, filesize, remoteSite, traceTimeentryUnix\
                               ORDER BY 10 DESC")\

In [9]:
df_ps.show(n=5)

+-------+-----------+--------------------+---------+------------+--------------------+---------+--------------------+-------------------+--------+
|account|clientState|             dataset|eventType|eventVersion|            filename| filesize|          remoteSite| traceTimeentryUnix|count(1)|
+-------+-----------+--------------------+---------+------------+--------------------+---------+--------------------+-------------------+--------+
|   null|       DONE|mc15_14TeV:mc15_1...|   get_sm|      pilot3|HITS.13102617._00...|971192562|  CERN-PROD_DATADISK|1.523785044031169E9|       3|
|   null|       DONE|mc15_13TeV.345055...|   put_sm|      pilot3|TXT.13720335._018...|    87744|       MWT2_DATADISK|1.523785041325712E9|       3|
|   null|       DONE|mc15_13TeV.345056...|   put_sm|      pilot3|EVNT.13720337._02...|  2508342|   BNL-OSG2_DATADISK|1.523785032822271E9|       3|
|   null|       DONE|mc15_14TeV:mc15_1...|   get_sm|      pilot3|HITS.13085786._00...|373000250|  CERN-PROD_DATADISK|1

In [10]:
# df_nulls = df_ps.where(col("account").isNull())

In [11]:
# df_nulls.show(n=5)

In [12]:
# replacing null values in column account with the value of column eventVersion of the same row
# df_ps2 = df_ps.withColumn("account",coalesce(df.account,df.eventVersion))
# df_ps2.show(n=5)

df_ps = df_ps.dropna()
df_ps.show(n=5)

+--------+------------------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+--------+
| account|       clientState|             dataset|eventType|eventVersion|            filename|  filesize|          remoteSite| traceTimeentryUnix|count(1)|
+--------+------------------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+--------+
|  aivina|              DONE|mc15_pPb8TeV.3611...| download|     1.14.11|AOD.13224270._000...|3986202718|  CERN-PROD_DATADISK| 1.52378503405265E9|       3|
|toyamaza|              DONE|mc16_13TeV.364179...| download|     1.14.11|DAOD_SUSY5.134628...| 853108798|TECHNION-HEP_DATA...|1.523785026267904E9|       3|
|toyamaza|              DONE|mc16_13TeV.364179...| download|     1.14.11|DAOD_SUSY5.134628...| 861521525|    NDGF-T1_DATADISK|1.523785043488475E9|       3|
|ggallard|              DONE|data17_13TeV.0034...| download|    

In [13]:
split_col = split(df_ps['dataset'], "\.")
df_ps = df_ps.withColumn('project', split_col.getItem(0))
df_ps = df_ps.withColumn('run_number', split_col.getItem(1))
df_ps = df_ps.withColumn('stream_name', split_col.getItem(2))
df_ps = df_ps.withColumn('prod_step', split_col.getItem(3))
df_ps = df_ps.withColumn('datatype', split_col.getItem(4))
df_ps = df_ps.withColumn('dataset_version', split_col.getItem(5))
df_ps.show(n=5)


+--------+-----------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+--------+------------+----------+--------------------+---------+----------+--------------------+
| account|clientState|             dataset|eventType|eventVersion|            filename|  filesize|          remoteSite| traceTimeentryUnix|count(1)|     project|run_number|         stream_name|prod_step|  datatype|     dataset_version|
+--------+-----------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+--------+------------+----------+--------------------+---------+----------+--------------------+
|toyamaza|       DONE|mc16_13TeV.364179...| download|     1.14.11|DAOD_SUSY5.134628...| 853108798|TECHNION-HEP_DATA...|1.523785026267904E9|       3|  mc16_13TeV|    364179|Sherpa_221_NNPDF3...|    deriv|DAOD_SUSY5|e5340_s3126_r1020...|
|  aivina|       DONE|mc15_pPb8TeV.3611...| download|   

### Perform One-Hot-encoding to make categorical data numerical (necessary for linear regression)

In [14]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
# https://spark.apache.org/docs/2.1.0/ml-features.html#onehotencoder


In [15]:

categoricalFeatures = ['account', 'clientState', 'dataset', 'eventType', 'eventVersion', 'filename', 'remoteSite', 'project', 'stream_name', 'prod_step', 'datatype'] # feature dataset version causes error: "Caused by: org.apache.spark.SparkException: StringIndexer encountered NULL value. To handle or skip NULLS, try setting StringIndexer.handleInvalid."

encoded = df_ps

for elem in categoricalFeatures:
    stringIndexer = StringIndexer(inputCol=elem, outputCol= elem + "Index")
    model = stringIndexer.fit(encoded)
    indexed = model.transform(encoded)
    
    if indexed.select(elem + 'Index').distinct().count() > 1:
        encoder = OneHotEncoder(inputCol= elem + "Index", outputCol= elem + "Vec")
        encoded = encoder.transform(indexed)

encoded.show(n=5)

+--------+-----------+--------------------+---------+------------+--------------------+----------+--------------------+-------------------+--------+------------+----------+--------------------+---------+----------+--------------------+------------+-----------------+----------------+--------------+------------+--------------------+-----------------+---------------+-------------+--------------------+---------------+----------------+------------+---------------+----------------+------------------+--------------+-------------+-------------+---------------+
| account|clientState|             dataset|eventType|eventVersion|            filename|  filesize|          remoteSite| traceTimeentryUnix|count(1)|     project|run_number|         stream_name|prod_step|  datatype|     dataset_version|accountIndex|       accountVec|clientStateIndex|clientStateVec|datasetIndex|          datasetVec|eventVersionIndex|eventVersionVec|filenameIndex|         filenameVec|remoteSiteIndex|   remoteSiteVec|project

In [16]:
encoded.createOrReplaceTempView("encoded")

In [17]:
# TODO: create labels by counting dataset download events => then set new label for these datasets ('cache' if more than X counts , else 'ncache')
# TODO: linear regression like here: https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning#basics

df_ps_counts = spark.sql("SELECT dataset, count(*) as hits \
                               FROM traces \
                               WHERE dataset != 'None' AND dataset != '' AND dataset != '[null]' AND (dataset LIKE 'data%' OR dataset LIKE 'mc%')\
                               GROUP BY  dataset \
                               ORDER BY 2 DESC")
df_ps_counts.show(n=5)

+--------------------+-------+
|             dataset|   hits|
+--------------------+-------+
|mc16_13TeV:mc16_1...|1640973|
|mc16_13TeV:mc16_1...| 985279|
|mc15_13TeV:mc15_1...| 796275|
|mc15_13TeV:mc15_1...| 791602|
|mc15_13TeV:mc15_1...| 465155|
+--------------------+-------+
only showing top 5 rows



In [18]:
df_ps_counts.createOrReplaceTempView("counts")

In [23]:
df_ps_merged = spark.sql("SELECT * \
                               FROM encoded \
                               JOIN counts ON encoded.dataset = counts.dataset")

In [24]:
df_ps_merged.show(n=5)

+-------+-----------+--------------------+---------+------------+--------------------+---------+--------------------+-------------------+--------+------------+----------+------------+---------+------------+-----------------+------------+---------------+----------------+--------------+------------+-------------------+-----------------+---------------+-------------+--------------------+---------------+----------------+------------+--------------+----------------+----------------+--------------+-------------+-------------+---------------+--------------------+----+
|account|clientState|             dataset|eventType|eventVersion|            filename| filesize|          remoteSite| traceTimeentryUnix|count(1)|     project|run_number| stream_name|prod_step|    datatype|  dataset_version|accountIndex|     accountVec|clientStateIndex|clientStateVec|datasetIndex|         datasetVec|eventVersionIndex|eventVersionVec|filenameIndex|         filenameVec|remoteSiteIndex|   remoteSiteVec|projectIndex

In [21]:
#encoded.join(df_ps_counts, ['dataset'], 'left_outer')
#encoded

In [26]:
df_ps_counts.count()

885651

In [27]:
encoded.count()

781484

In [ ]:
df_ps_merged.count()